--- 
Project for the course in Microeconometrics | Summer 2021, M.Sc. Economics, Bonn University | [Philipp Schreiber](https://github.com/pcschreiber1)

# Replication of Henderson, Storeygard, Deichmann (2017) :
## Has climate change driven urbanization in Africa? <a class="tocSkip">   
    
    
---

<div class="alert alert-block alert-info">
    <h5><b>Downloading and viewing this notebook:</b></h5> <p> The original paper, as well as the data and code provided by the authors can be accessed <a href="https://doi.org/10.1016/j.jdeveco.2016.09.001">here</a> </p></div>

## Content:
Extension: 
 - Spatial SUTVA/ <span style="color:orange">**NOTE**:</span> How large are the districts (important for whetehr SUTVA is reasonable or not)
 - Spatially explicit counterfactual framework
     - using exploration of SUTVA and spatial patters
         - spatial autocorrelation: Moran Plot and Moran's I
     - implementing (lagged dependent, lagged 

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker
import statsmodels.formula.api as smf
import seaborn as sns

#For spatial analysis
import geopandas as gpd
import shapely.geometry as geom
import libpysal as lp #For spatial weights

from pysal.viz import splot #exploratory analysis
#from splot.esda import plot_moran #exploratory analysis
from pysal.explore import esda #exploratory analysis
from pysal.model import spreg #For spatial regression

pd.options.display.float_format = "{:,.2f}".format

In [ ]:
from auxiliary.data_import import *
from auxiliary.plots import *

In [1]:
#map_countries()
regiondata = get_shapefile()

NameError: name 'get_shapefile' is not defined

In [ ]:
# The explanatory variables of interest
a = ["ADsm0_1moistu",
     "ADsm0_2moistu",
     "ADsm0_3moistu",
     "ADsm0_4moistu",
     "extent_agE_ADsm0_2moistu",
     "ADurbfrac",
     "extent_agE",
     "extent_agH",
     "primate"]